# Fase 2 - Incorporacion de IBM Model en un dispositivo remoto

Inicialmente realizamos las instalaciones necesarias. Previo a esto se tiene que crear un bucket en IBM Cloud Storage con el fin de tener un repositorio en la nube. Dicho repositorio contiene el modelo utilizado en esta actividad. Para mayor información del modelo acudir a:
- *../Fase2-AIML.ipynb*

In [2]:
pip install ibm-cos-sdk


  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ---------------------------------------- 0.0/1.1 MB ? eta -:--:--
     ---------------------------------------- 1.1/1.1 MB 8.9 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for ibm-cos-sdk: filename=ibm_cos_sdk-2.14.0-py3-none-any.whl size=77253 sha256=dc873ec4068564084ca87697a2f3db25489ef62d9780a630c5d3a53fe7a6a036
  Stored in directory: c:\users\usuario\appdata\local\pip\cache\wheels\f4\77\da\bf8b32e81b9ce22916583537d428c8713c9232931d7a74408d
  Created wheel for ibm-cos-sdk-core: filename=ibm_cos_sdk_core-2.14.0-py3-none-any.whl size=661739 sha256=5b56b172948a1beef8ee9f369b7ec52474b81ccbb96a92c9f7ceb8667a554a43
  Stored in directory: c:\users\usuario\appdata\local\pip\cache\wheels\6b\43\bd\

In [1]:
pip install boto3

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
aiobotocore 2.12.3 requires botocore<1.34.70,>=1.34.41, but you have botocore 1.35.76 which is incompatible.



   ---------------------------------------- 0.0/13.2 MB ? eta -:--:--
   ---------------- ----------------------- 5.5/13.2 MB 30.7 MB/s eta 0:00:01
   ---------------------------------------- 13.2/13.2 MB 37.8 MB/s eta 0:00:00
  Attempting uninstall: botocore
    Found existing installation: botocore 1.34.69
    Uninstalling botocore-1.34.69:
      Successfully uninstalled botocore-1.34.69


In [5]:
pip install ibm_cos_sdk

Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install ibm_cloud_sdk_core

Note: you may need to restart the kernel to use updated packages.


Una vez realizadas las instalaciones, se añaden las bibliotecas necesarias para acceder al storage en IBM. Al crear dicho almacenamiento, IBM provee servicios de seguridad de credenciales, dichas son necesarias de almacenar en algún archivo, que se añadirá a esta entrega y se puede encontrar en:
- *'../docs/key_services.txt'*

Solo haría falta modificar los requerimientos y conectar al bucket.

Se carga el modelo y este podrá ser utilizado.

In [47]:
import ibm_boto3
from ibm_botocore.client import Config
import xgboost as xgb
import pandas as pd

In [58]:
data = pd.read_csv('Fase2_ML - Data.csv')
data = data[['ALTO', 'ANCHO', 'LARGO', 'PESO_ASIGNADO', 'TEMPERATURA_ALMACENAMIENTO_NUM', 'LUGAR_PROCEDENCIA_WOE', 'TIPO_MANIPULACION_NUM', 'TEMPERATURA_ALMACENAMIENTO_NUM']].values

In [35]:
api_key = "JaLEXB17Mg0BwhBHG6G8ca8iMVKlYApEpUjCCzdesPDf"  # Get this from your IBM Cloud service credentials
service_instance_id = 'crn:v1:bluemix:public:iam-identity::a/bc5cdf979df746968857c62ed0ba815a::serviceid:ServiceId-bf2ee9f6-10d0-44a3-9665-51805c00ac5d'  # Get this from your IBM Cloud service credentials
bucket_name = 'fase2-ai-ml-mart'
model_file_name = 'Fase2_ML - Model.bst'  # Replace with your actual model file name
local_model_path = 'Fase2_ML - Model_Local.bst'

In [17]:
cos = ibm_boto3.client('s3',
                  ibm_api_key_id=api_key,
                  ibm_service_instance_id=service_instance_id,
                  config=Config(signature_version='oauth'),
                  endpoint_url='https://s3.us-south.cloud-object-storage.appdomain.cloud')  # Replace with the correct endpoint for your region

In [37]:
cos.download_file(bucket_name, model_file_name, local_model_path)

In [41]:
model = xgb.Booster()
model.load_model(local_model_path)

In [60]:
prediction = model.predict(xgb.DMatrix(data))